In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

# Import all files to notebook

In [2]:
train = pd.read_csv(r'..\data\house prediction\train.csv',index_col = 'Id')
test = pd.read_csv(r'..\data\house prediction\test.csv',index_col = 'Id')
ans = pd.read_csv(r'..\data\house prediction\sample_submission.csv',index_col = 'Id')

In [3]:
# test = pd.merge(test,ans,on= 'Id')

In [4]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1460 entries, 1 to 1460
Data columns (total 80 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   MSSubClass     1460 non-null   int64  
 1   MSZoning       1460 non-null   object 
 2   LotFrontage    1201 non-null   float64
 3   LotArea        1460 non-null   int64  
 4   Street         1460 non-null   object 
 5   Alley          91 non-null     object 
 6   LotShape       1460 non-null   object 
 7   LandContour    1460 non-null   object 
 8   Utilities      1460 non-null   object 
 9   LotConfig      1460 non-null   object 
 10  LandSlope      1460 non-null   object 
 11  Neighborhood   1460 non-null   object 
 12  Condition1     1460 non-null   object 
 13  Condition2     1460 non-null   object 
 14  BldgType       1460 non-null   object 
 15  HouseStyle     1460 non-null   object 
 16  OverallQual    1460 non-null   int64  
 17  OverallCond    1460 non-null   int64  
 18  YearBuil

In [8]:
def feature_select(data):
    null = []
    for i in train.columns:
        val = pd.value_counts(train[i].isna())
        if val[0]/len(train) <= 0.6:
            null.append(i)
    data.drop(columns=null,inplace = True)
    data.fillna(data.median(),inplace = True)
    data.dropna(axis = 0,inplace = True)
    return data

In [9]:
train_clean = feature_select(train)

In [10]:
object_col = train_clean.select_dtypes(include='object').columns
object_index = train_clean.select_dtypes(include='object').index

In [11]:
object_index

Int64Index([   1,    2,    3,    4,    5,    6,    7,    8,    9,   10,
            ...
            1448, 1449, 1452, 1453, 1455, 1456, 1457, 1458, 1459, 1460],
           dtype='int64', name='Id', length=1338)

In [12]:
from sklearn.preprocessing import LabelEncoder
def encoder(data):
    encoder_data = []
    le = LabelEncoder()
    for i in data.columns:
        encoder_data.append(le.fit_transform(data[i]))
    return np.transpose(encoder_data)
#     return pd.DataFrame(np.reshape(np.array(encoder_data),(row,columns)),columns=data_col,index=data_index) 

In [13]:
def combine(data):
    col = data.columns
    ind = data.index
    data_encoder = pd.DataFrame(encoder(data[col]),columns=col,index=ind)
    return pd.merge(data.drop(columns = col),data_encoder,how='inner',on = 'Id')

In [14]:
train_complete = combine(train_clean)

In [15]:
train_complete.head(3)

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,LotShape,LandContour,Utilities,LotConfig,LandSlope,...,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
Id,,,,,,,,,,,,,,,,,,,,,
1,5,3,35,287,1,3,3,0,4,0,...,0,0,0,0,0,1,2,8,4,371
2,0,3,50,448,1,3,3,0,2,0,...,0,0,0,0,0,4,1,8,4,303
3,5,3,38,641,1,0,3,0,4,0,...,0,0,0,0,0,8,2,8,4,400


In [17]:
from sklearn.metrics.pairwise import cosine_similarity
similarity = cosine_similarity(train_complete)

In [163]:
def match_house(house_id):
    a = sorted(dict(enumerate(similarity[house_id])).items(), key=lambda x: x[1], reverse=True)[1:4]
    return a

In [164]:
b = match_house(89)

In [165]:
print("I recommend you the house number {} ".format(b[0][0]))

I recommend you the house number 92 


In [159]:
# Build Application to recommend House
def match_house(house_id):
    a = sorted(dict(enumerate(similarity[house_id])).items(), key=lambda x: x[1], reverse=True)[1:4]
    return a

0.9992007760895265

In [168]:
hid = input("Please enter favourite house : \n")
house = match_house(int(hid))
print("I recommend you the house number {} ".format(house[0][0]))

Please enter favourite house : 
89
I recommend you the house number 92 
